In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

/Users/nicole/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load in data

In [2]:
# file_path_trainC = "widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.csv"
# train_cat = pd.read_csv(file_path_trainC)

# file_path_trainFCM = "widsdatathon2025/TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv"
# train_FCM = pd.read_csv(file_path_trainFCM)

# file_path_trainQ = "widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.csv"
# train_Quant = pd.read_csv(file_path_trainQ)

# file_path_trainS = "widsdatathon2025/TRAIN/TRAINING_SOLUTIONS.csv"
# train_Solutions = pd.read_csv(file_path_trainS)
# train_Solutions.drop(columns = ['Unnamed: 3', 'Unnamed: 4'], inplace = True)

file_path_trainC = "widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx"
train_cat = pd.read_excel(file_path_trainC)

file_path_trainFCM = "widsdatathon2025/TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv"
train_FCM = pd.read_csv(file_path_trainFCM)

file_path_trainQ = "widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx"
train_Quant = pd.read_excel(file_path_trainQ)

file_path_trainS = "widsdatathon2025/TRAIN/TRAINING_SOLUTIONS.xlsx"
train_Solutions = pd.read_excel(file_path_trainS)

In [3]:
file_path_testC = "widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx"
test_cat = pd.read_excel(file_path_testC)

file_path_testFCM = "widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv"
test_FCM = pd.read_csv(file_path_testFCM)

file_path_testQ = "widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx"
test_Quant = pd.read_excel(file_path_testQ)

## Combine train and test categorical data sets in order to get summative one hot encoding

In [4]:
train_cat = train_cat.drop(columns = 'Basic_Demos_Enroll_Year')

In [5]:
train_shape = train_cat.shape

for col in train_cat.select_dtypes(include = 'number').columns:
    train_cat[col] = train_cat[col].astype('Int64')
    train_cat[col] = train_cat[col].astype('category')
    
for col in test_cat.select_dtypes(include = 'number').columns:
    test_cat[col] = test_cat[col].astype('Int64')
    test_cat[col] = test_cat[col].astype('category')
    
cat_train_test = pd.concat([train_cat, test_cat], axis = 0).reset_index(drop = True)
columns_to_encode = cat_train_test.columns[1:].tolist()
cat_train_test_final = cat_train_test.drop(columns = columns_to_encode)

for col in cat_train_test.columns[1:]:
    encoded = pd.get_dummies(cat_train_test[col], prefix = col)
    cat_train_test_final = pd.concat([cat_train_test_final, encoded], axis = 1)

cat_train_test_final.head()

,participant_id,Basic_Demos_Study_Site_1,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,Basic_Demos_Study_Site_5,PreInt_Demos_Fam_Child_Ethnicity_0,PreInt_Demos_Fam_Child_Ethnicity_1,PreInt_Demos_Fam_Child_Ethnicity_2,PreInt_Demos_Fam_Child_Ethnicity_3,...,Barratt_Barratt_P2_Occ_25,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Enroll_Year_2021,Basic_Demos_Enroll_Year_2022,Basic_Demos_Enroll_Year_2023
0,00aIpNTbG5uh,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00fV0OyyoLfw,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,04X1eiS79T4B,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,05ocQutkURd6,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,06YUNBA9ZRLq,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Split back into training and testing sets and combine with quantitative and FCM data

In [6]:
cat_train_final = cat_train_test_final.iloc[0:train_shape[0]]
cat_test_final = cat_train_test_final.iloc[train_shape[0]:]

In [7]:
logreg_sex = pd.merge(train_FCM, train_Quant, on = 'participant_id')
train_df = pd.merge(cat_train_final, logreg_sex, on = 'participant_id')

# ensure it looks accurate
print(train_df.shape)
train_df.head()

(1213, 19981)


,participant_id,Basic_Demos_Study_Site_1,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,Basic_Demos_Study_Site_5,PreInt_Demos_Fam_Child_Ethnicity_0,PreInt_Demos_Fam_Child_Ethnicity_1,PreInt_Demos_Fam_Child_Ethnicity_2,PreInt_Demos_Fam_Child_Ethnicity_3,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,00aIpNTbG5uh,0,0,0,1,0,0,1,0,0,...,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
1,00fV0OyyoLfw,1,0,0,0,0,1,0,0,0,...,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,NaN
2,04X1eiS79T4B,1,0,0,0,0,0,1,0,0,...,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
3,05ocQutkURd6,1,0,0,0,0,0,0,0,1,...,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
4,06YUNBA9ZRLq,1,0,0,0,0,1,0,0,0,...,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574


In [8]:
test_cat_FCM = pd.merge(cat_test_final, test_FCM, on = 'participant_id')
test_df = pd.merge(test_cat_FCM, test_Quant, on = 'participant_id')

# ensure it looks accurate
print(test_df.shape)
test_df.head()

(304, 19981)


,participant_id,Basic_Demos_Study_Site_1,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,Basic_Demos_Study_Site_5,PreInt_Demos_Fam_Child_Ethnicity_0,PreInt_Demos_Fam_Child_Ethnicity_1,PreInt_Demos_Fam_Child_Ethnicity_2,PreInt_Demos_Fam_Child_Ethnicity_3,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,Cfwaf5FX7jWK,0,0,0,1,0,1,0,0,0,...,2.0,12.0,3.0,9.0,2.0,7.0,3.0,0.0,8.0,8.992813
1,vhGrzmvA3Hjq,0,0,0,1,0,1,0,0,0,...,2.0,16.0,8.0,5.0,7.0,3.0,11.0,3.0,9.0,12.324093
2,ULliyEXjy4OV,0,0,0,1,0,1,0,0,0,...,1.0,7.0,1.0,6.0,1.0,5.0,1.0,0.0,9.0,7.770933
3,LZfeAb1xMtql,0,0,0,1,0,1,0,0,0,...,4.0,15.0,4.0,10.0,8.0,6.0,5.0,1.0,6.0,9.304814
4,EnFOUv0YK1RG,0,0,0,1,0,0,0,1,0,...,2.0,18.0,6.0,12.0,5.0,10.0,6.0,0.0,10.0,8.261350


## Fill NaN values and missings

In [9]:
# print(logreg_sex.isna().sum())

# logreg_sex.fillna({'MRI_Track_Age_at_Scan':logreg_sex['MRI_Track_Age_at_Scan'].mean()}, inplace = True)

# print(logreg_sex.isna().sum().sum()) # should now be zero

In [10]:
# print(train_df.isna().sum())

# train_df.fillna({'MRI_Track_Age_at_Scan':train_df['MRI_Track_Age_at_Scan'].mean()}, inplace = True)

# print(train_df.isna().sum().sum()) # should now be zero

In [11]:
for col in logreg_sex.columns[1:]:
    logreg_sex.fillna({col: logreg_sex[col].median()}, inplace = True)

for col in train_df.columns[1:]:
    train_df.fillna({col: train_df[col].median()}, inplace = True)

## Logistic Regression for Feature Selection

**We will first use L1 regularization with Sex as the target.**

In [12]:
X_train_sex, X_val_sex, y_train_sex, y_val_sex = train_test_split(logreg_sex.drop(columns = ['participant_id']), 
                                                                  train_Solutions['Sex_F'], test_size = .2)

In [13]:
sex_model = LogisticRegression(penalty = 'l1', solver = 'liblinear')
weights = [2 if i == 1 else 1 for i in y_train_sex]
sex_model.fit(X_train_sex, y_train_sex, weights)
accuracy_score(y_val_sex, sex_model.predict(X_val_sex))

0.5349794238683128

In [14]:
np.mean(cross_val_score(sex_model, X_train_sex, y_train_sex, cv = 5))

0.5257731958762888

**We will now use L1 regularization with ADHD_Outcome as the target**

In [15]:
X_train_adhd, X_val_adhd, y_train_adhd, y_val_adhd = train_test_split(train_df.drop(columns = 'participant_id'), 
                                                                      train_Solutions['ADHD_Outcome'], test_size = .2)

In [16]:
adhd_model = LogisticRegression(penalty = 'l1', solver = 'liblinear')
adhd_model.fit(X_train_adhd, y_train_adhd)

LogisticRegression(penalty='l1', solver='liblinear')

In [17]:
adhd_model.score(X_val_adhd, y_val_adhd)
np.mean(cross_val_score(adhd_model, X_train_adhd, y_train_adhd, cv = 5))

0.5989690721649484

## Evaluate

In [18]:
# ORIGINAL VALUES BELOW
# Cross-validation scores for each fold: [0.82098765 0.78600823 0.69753086 0.64669421 0.33471074]
# Mean CV score: 0.657186341529776

# CV w/ intersection of features from logistic regression
# Cross-validation scores for each fold: [0.78806584 0.76748971 0.69341564 0.66322314 0.45041322]
# Mean CV score: 0.6725215114104003

# CV w/ union of features from logistic regression
# Cross-validation scores for each fold: [0.83950617 0.83333333 0.74074074 0.70041322 0.43181818]
# Mean CV score: 0.7091623303744514

# CV for partial data logreg sex and all data logreg adhd separate decision trees
# Cross-validation scores for each fold: [0.64948454 0.62886598 0.63917526 0.65463918 0.63402062]
# Mean CV score: 0.6412371134020618
# Cross-validation scores for each fold: [0.80927835 0.85051546 0.83505155 0.79381443 0.80412371]
# Mean CV score: 0.8185567010309278

# CV for partial data logreg sex and all data for adhd, one decision tree
# Cross-validation scores for each fold: [0.80864198 0.78600823 0.7037037  0.65082645 0.40289256]
# Mean CV score: 0.6704145835458967

# CV for encoded random forest, partial data for sex, all for adhd
# Cross-validation scores for each fold: [0.50515464 0.50515464 0.48453608 0.49484536 0.53092784]
# Mean CV score: 0.5041237113402062

# CV for multioutput random forest, partial data for sex, all for adhd
# Cross-validation scores for each fold: [0.72680412 0.71907216 0.70103093 0.71649485 0.68814433]
# Mean CV score: 0.7103092783505154

# CV for separate random forests, adhd, gender
# Cross-validation scores for each fold: [0.80412371 0.77319588 0.80412371 0.80927835 0.84020619]
# Mean CV score: 0.8061855670103093
# Cross-validation scores for each fold: [0.67010309 0.68041237 0.67010309 0.67525773 0.67010309]
# Mean CV score: 0.6731958762886598